In [2]:
import numpy as np
import pandas as pd

In [3]:
YSL_export_16096040 = pd.read_csv("YSL_export_16096040.csv")
YSL_export2_16096121 = pd.read_csv("YSL_export2_16096121.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'YSL_export_16096040.csv'

In [ ]:
df = pd.concat(
    [YSL_export_16096040, YSL_export2_16096121],
    axis=0,         # 按行拼接
    ignore_index=True  # 重新生成索引
)
print(f"✅ 合并完成，共 {len(df)} 条数据。")
df.to_csv("/root/ysl_project/df")

✅ 合并完成，共 515791 条数据。


In [ ]:
cleaned_df = df[["id", "url", "标题", "内容", "话题", "发表时间", "总互动量", "转发数", "评论数", "点赞数", "收藏数", "用户id", "小红书号", "用户属地", "性别", "出生年份", "关键词", "用户地址"]]

In [ ]:
# 🔹 拼接内容、话题、关键词 → 新列 “完整内容”
cleaned_df["完整内容"] = (
    cleaned_df["内容"].fillna("") + "。" +
    "话题：" + cleaned_df["话题"].fillna("") + "。" +
    "关键词：" + cleaned_df["关键词"].fillna("")
).str.strip()


/tmp/ipykernel_38694/3409648221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["完整内容"] = (


In [ ]:
import re
import pandas as pd

def clean_text_minimal(text: str) -> str:
    """删除 emoji 和邮箱"""
    if pd.isna(text):
        return text
    s = str(text)

    # 去邮箱
    s = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '', s)

    # 去 emoji（匹配所有非BMP表情符号）
    emoji_pattern = re.compile("[\U00010000-\U0010FFFF]", flags=re.UNICODE)
    s = emoji_pattern.sub("", s)

    return s.strip()

# 示例应用
text_cols = ["内容","标题", "完整内容"]  # 你要清洗的列名
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_text_minimal)

print("✅ 文本清洗完成！示例：")
print(cleaned_df[text_cols])

✅ 文本清洗完成！示例：
                                                       内容  \
0       Y家的麂皮棕配上大衣和毛衣真的高级松弛感拉满～ 包身采用奢华轻便菱形绗缝明线，饱满丝滑，触感...   
1       #ysl圣罗兰 #YSL圣罗兰 #圣罗兰 #连体裤 #设计感连衣裙 #露背装 #小黑裙 #露...   
2       YSL板材微翘猫眼款🕶️ 板材前框加上金属镜腿，ysl交织logo 镜片渐变色，超级好看时尚...   
3       #ysl #超模 #骨子里的优雅 #时尚女魔头 #时尚 #时尚界宠儿 #时尚资讯 #寻找秀场...   
4       ysl全品类参加活动， 力度开店以来最大一次 抓住机会哦 #ysl #新款分享 #skp活动...   
...                                                   ...   
515786  后悔没早点入，上身真的太好看实用啦，虽然包包是替替，但日常使用超满足#ysl #ysl圣罗兰...   
515787                  没有女孩子能抗拒一只银色的YSL #圣罗兰 #中古包vintage   
515788  #包包分享 #包包不重样 #最爱的包包 #ysl包包 #ysl #ysl圣罗兰 #YSL圣罗...   
515789                                            #留学 #uk   
515790  前前后后去试了jimmychoo/Christian Louboutin/ Roger Vi...   

                         标题                                               完整内容  
0         来了｜秋天得必备麂皮𝐈𝐂𝐀𝐑𝐈𝐍𝐎  Y家的麂皮棕配上大衣和毛衣真的高级松弛感拉满～ 包身采用奢华轻便菱形绗缝明线，饱满丝滑，触感...  
1                     露背连体裤  #ysl圣罗兰 #YSL圣罗兰 #圣罗兰 #连体裤 #设计感连衣裙 #露背装 #小黑裙 #露...  
2          

In [ ]:
# 1️⃣ 计算总行数
total_rows = len(cleaned_df)

# 2️⃣ 过滤掉内容长度小于 15 的行
cleaned_df = cleaned_df[cleaned_df['内容'].astype(str).str.len() >= 15].copy()

# 3️⃣ 计算被删掉的比例
removed_ratio = 1 - len(cleaned_df) / total_rows

print(f"✅ 原始行数: {total_rows}")
print(f"✅ 保留行数: {len(cleaned_df)}")
print(f"📉 删除比例: {removed_ratio:.2%}")

✅ 原始行数: 515791
✅ 保留行数: 493843
📉 删除比例: 4.26%


In [ ]:
# 定义一个函数：去除标点符号、特殊字符
def clean_text(text):
    if pd.isna(text):
        return ""
    # 去掉所有中文、英文标点符号、表情、特殊符号，只保留中英文、数字和空格
    text = re.sub(r'[^\w\s\u4e00-\u9fff]', '', str(text))
    # 去除多余空格
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 2️⃣ 清理“完整内容”列
cleaned_df['完整内容_清理后'] = cleaned_df['完整内容'].apply(clean_text)

# 3️⃣ 合并“内容”和清理后的“完整内容”
cleaned_df['合并内容'] = cleaned_df['标题'].fillna('') + ' ' + cleaned_df['完整内容_清理后']

# 4️⃣ 再次清理合并后的文本（防止重复标点或空格）
cleaned_df['合并内容'] = cleaned_df['合并内容'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# 5️⃣ 输出结果
print(cleaned_df[['内容', '完整内容', '完整内容_清理后', '合并内容']].head())

                                                  内容  \
0  Y家的麂皮棕配上大衣和毛衣真的高级松弛感拉满～ 包身采用奢华轻便菱形绗缝明线，饱满丝滑，触感...   
1  #ysl圣罗兰 #YSL圣罗兰 #圣罗兰 #连体裤 #设计感连衣裙 #露背装 #小黑裙 #露...   
2  YSL板材微翘猫眼款🕶️ 板材前框加上金属镜腿，ysl交织logo 镜片渐变色，超级好看时尚...   
3  #ysl #超模 #骨子里的优雅 #时尚女魔头 #时尚 #时尚界宠儿 #时尚资讯 #寻找秀场...   
4  ysl全品类参加活动， 力度开店以来最大一次 抓住机会哦 #ysl #新款分享 #skp活动...   

                                                完整内容  \
0  Y家的麂皮棕配上大衣和毛衣真的高级松弛感拉满～ 包身采用奢华轻便菱形绗缝明线，饱满丝滑，触感...   
1  #ysl圣罗兰 #YSL圣罗兰 #圣罗兰 #连体裤 #设计感连衣裙 #露背装 #小黑裙 #露...   
2  YSL板材微翘猫眼款🕶️ 板材前框加上金属镜腿，ysl交织logo 镜片渐变色，超级好看时尚...   
3  #ysl #超模 #骨子里的优雅 #时尚女魔头 #时尚 #时尚界宠儿 #时尚资讯 #寻找秀场...   
4  ysl全品类参加活动， 力度开店以来最大一次 抓住机会哦 #ysl #新款分享 #skp活动...   

                                            完整内容_清理后  \
0  Y家的麂皮棕配上大衣和毛衣真的高级松弛感拉满 包身采用奢华轻便菱形绗缝明线饱满丝滑触感细腻软...   
1  ysl圣罗兰 YSL圣罗兰 圣罗兰 连体裤 设计感连衣裙 露背装 小黑裙 露背裙 气质小黑裙...   
2  YSL板材微翘猫眼款 板材前框加上金属镜腿ysl交织logo 镜片渐变色超级好看时尚 亚洲版...   
3  ysl 超模 骨子里的优雅 时尚女魔头 时尚 时尚界宠儿 时尚资讯 寻找秀场上的美 圣罗兰 ...   
4  ysl全品类参加活动 力度开店以来最大一次 抓住机会哦 ysl 新款分享 skp活动 

In [ ]:
cleaned_df.to_csv("cleaned_df_v1.csv")

In [ ]:
# 计算每列空值比例
null_summary = (
    cleaned_df.isna()
    .mean()
    .sort_values(ascending=False)
    .to_frame("缺失比例")
    .assign(非空比例=lambda x: 1 - x["缺失比例"])
)

# 转换为百分比显示
null_summary["缺失比例"] = (null_summary["缺失比例"] * 100).round(2).astype(str) + "%"
null_summary["非空比例"] = (null_summary["非空比例"] * 100).round(2).astype(str) + "%"

print("✅ 各列缺失情况：")
display(null_summary.head(20))  # 显示前 20 列

✅ 各列缺失情况：


,缺失比例,非空比例
出生年份,93.19%,6.81%
用户地址,49.2%,50.8%
性别,43.04%,56.96%
用户属地,32.43%,67.57%
转发数,29.77%,70.23%
收藏数,29.67%,70.33%
评论数,28.48%,71.52%
小红书号,15.41%,84.59%
话题,7.8%,92.2%
标题,3.45%,96.55%


# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ✅ 确保中文可显示
plt.rcParams['font.sans-serif'] = ['PingFang SC', 'Microsoft YaHei', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("==== 🧾 Dataset Overview ====")
print(f"Total rows: {len(cleaned_df)}")
print(f"Total columns: {len(cleaned_df.columns)}\n")

# =============================
# 1️⃣ ID Uniqueness
# =============================
total = len(cleaned_df)
unique = cleaned_df['post_id'].nunique()
duplicates = total - unique
print(f"🆔 ID Uniqueness Check: Total {total}, Unique IDs {unique}, Duplicates {duplicates}.")
if duplicates > 0:
    dup_ids = cleaned_df['post_id'].value_counts().loc[lambda x: x > 1]
    print("⚠️ Example of Duplicate IDs:\n", dup_ids.head())

# =============================
# 2️⃣ Post Time Distribution
# =============================
if 'post_time' in cleaned_df.columns:
    cleaned_df['post_time'] = pd.to_datetime(cleaned_df['post_time'], errors='coerce')
    print("\n🕒 Post Time Range:")
    print(f"Earliest: {cleaned_df['post_time'].min()}  Latest: {cleaned_df['post_time'].max()}")
    print("📈 Posts per Month:")
    cleaned_df['Month'] = cleaned_df['post_time'].dt.to_period('M')
    post_counts = cleaned_df['Month'].value_counts().sort_index()
    display(post_counts.head(10))
    plt.figure(figsize=(10,4))
    post_counts.plot(kind='bar', color='teal', title='Monthly Post Count')
    plt.xlabel('Month')
    plt.ylabel('Number of Posts')
    plt.show()

# =============================
# 3️⃣ Engagement Metrics with Outlier Handling
# =============================

eng_col_map = {
    'interaction_times': 'Interaction Times',
    'shares': 'Shares',
    'comments': 'Comments',
    'likes': 'Likes',
    'saves': 'Saves'
}

def plot_with_outlier_split(df, col):
    if col not in df.columns:
        print(f"⚠️ Column {col} not found.")
        return
    if not np.issubdtype(df[col].dropna().dtype, np.number):
        print(f"⚠️ Column {col} is not numeric, skipping.")
        return

    col_en = eng_col_map.get(col, col)
    q95 = df[col].quantile(0.95)
    qmax = df[col].max()
    num_outliers = (df[col] > q95).sum()

    print(f"\n📊 {col_en} Distribution:")
    print(f"95th percentile = {q95:.2f}, max = {qmax:.2f}, outliers = {num_outliers} ({num_outliers/len(df)*100:.2f}%)")

    plt.figure(figsize=(8,4))
    sns.histplot(df[df[col] <= q95][col], bins=50, color='skyblue', edgecolor='black')
    plt.title(f'{col_en} Distribution (≤ 95th percentile)')
    plt.xlabel(col_en)
    plt.ylabel('Count')
    plt.show()

    if num_outliers > 0:
        plt.figure(figsize=(5,3))
        sns.boxplot(x=df[df[col] > q95][col], color='salmon')
        plt.title(f'{col_en} Outliers (>95th percentile)')
        plt.xlabel(col_en)
        plt.show()
    else:
        print(f"✅ No outliers beyond 95th percentile for {col_en}")

engagement_cols = ['interaction_times', 'shares', 'comments', 'likes', 'saves']
for col in engagement_cols:
    if col in cleaned_df.columns:
        plot_with_outlier_split(cleaned_df, col)

# =============================
# 4️⃣ User Location Distribution
# =============================
if 'user_location' in cleaned_df.columns:
    top_location = cleaned_df['user_location'].value_counts(dropna=True).head(10)
    print("\n🌍 Top 10 User Locations:")
    display(top_location)
    plt.figure(figsize=(8,4))
    top_location.plot(kind='bar', color='coral')
    plt.title('Top 10 User Locations')
    plt.xlabel('Region')
    plt.ylabel('User Count')
    plt.show()

# =============================
# 5️⃣ Gender Distribution
# =============================
if 'gender' in cleaned_df.columns:
    print("\n🚻 Gender Distribution:")

    cleaned_df['gender'] = cleaned_df['gender'].fillna('Unknown')
    cleaned_df.loc[cleaned_df['gender'].isin(['未知', '未填写', '不明', 'Unknown']), 'gender'] = 'Unknown'

    gender_counts = cleaned_df['gender'].value_counts(dropna=False)
    display(gender_counts)

    plt.figure(figsize=(4,4))
    gender_counts.plot(kind='pie', autopct='%.1f%%', colors=['pink','lightblue','gray'])
    plt.title('Gender Distribution')
    plt.ylabel('')
    plt.show()

# =============================
# 6️⃣ Birth Year Distribution (showing middle 90% + outliers)
# =============================
if 'birth_year' in cleaned_df.columns:
    print("\n👶 Birth Year Distribution (with outlier visualization):")
    
    cleaned_df['birth_year'] = pd.to_numeric(cleaned_df['birth_year'], errors='coerce')
    birth_series = cleaned_df['birth_year'].dropna()

    q0025 = birth_series.quantile(0.025)
    q0975 = birth_series.quantile(0.975)

    lower_outliers = birth_series[birth_series < q0025]
    middle_values = birth_series[(birth_series >= q0025) & (birth_series <= q0975)]
    upper_outliers = birth_series[birth_series > q0975]

    print(f"2.5th percentile = {q0025:.0f}, 97.5th percentile = {q0975:.0f}")
    print(f"Lower outliers: {len(lower_outliers)} ({len(lower_outliers)/len(birth_series)*100:.2f}%)")
    print(f"Upper outliers: {len(upper_outliers)} ({len(upper_outliers)/len(birth_series)*100:.2f}%)")

    plt.figure(figsize=(8,4))
    sns.histplot(middle_values, bins=30, color='gold', edgecolor='black')
    plt.title('Birth Year Distribution (Middle 95%)')
    plt.xlabel('Birth Year')
    plt.ylabel('User Count')
    plt.show()

    if len(lower_outliers) > 0:
        plt.figure(figsize=(5,3))
        sns.boxplot(x=lower_outliers, color='skyblue')
        plt.title('Lower 5% Birth Year Outliers')
        plt.xlabel('Birth Year')
        plt.show()

    if len(upper_outliers) > 0:
        plt.figure(figsize=(5,3))
        sns.boxplot(x=upper_outliers, color='salmon')
        plt.title('Upper 5% Birth Year Outliers')
        plt.xlabel('Birth Year')
        plt.show()

# =============================
# 7️⃣ User Address Samples
# =============================
if 'user_address' in cleaned_df.columns:
    print("\n🏠 User Address Samples:")
    sample_addr = cleaned_df['user_address'].dropna().head(10)
    print(sample_addr)
    print(f"Non-null ratio: {(cleaned_df['user_address'].notna().mean()*100):.2f}%")

==== 🧾 Dataset Overview ====


NameError: name 'cleaned_df' is not defined